## B6. Extract water mask

#### 0. Load required libraries,  area dependant constants and utility functions:

In [6]:
from os import listdir
from os.path import isfile, join
import numpy as np
import seaborn as sns

# EOLearn libraries:
from eolearn.core import EOTask, EOPatch, LinearWorkflow, LoadTask, SaveTask, FeatureType, EOExecutor
from eolearn.core import OverwritePermission

# Add to python path parent dictionary
import sys
sys.path.append("../../")

# load site dependant constants (HERE YOU CAN CHOOSE DIFFERENT LOCATION)
from aoi_sites import upe_promice_area as site

# load utility functions
from utils import io_functions as io_utils
from utils import plot_functions as plot_utils

#### 1. Set up and execute EOWorkflow

In [7]:
# defining eo_task for water feature extraction
import numpy.ma as ma
class StaticThreshiolding(EOTask):   
    """
    The tasks applies a static threshold to produce a data mask:
    MASK = 1 if > x else 0
    """
    def __init__(self, mask_name, feature_name, threshold):
        self.feature_name = feature_name
        self.mask_name = mask_name
        self.threshold = threshold
        
    def execute(self, eopatch):
        band_a = eopatch.data[self.feature_name][..., 0]
        mask = ma.masked_where(self.threshold <= band_a , band_a)
        # if the mask is empty it produces an one value False - it is beeter to have a matrix of False for consistency
        if mask.mask.any() == False:
            new_mask = np.zeros(mask.shape, dtype=bool)
            mask.mask = new_mask
            print('Created new mask of zeros, bc there was no water feature detected')
            
        eopatch.add_feature(FeatureType.MASK, self.mask_name, mask.mask[..., np.newaxis])
        return eopatch

#### 2. Set up and execute EOWorkflow

In [8]:
# creating eo tasks producing water mask from NDWI_ICE (based on blue/red NDI)
static_threshol_30_eotask = StaticThreshiolding('WATER_MASK_ICE_ST_030', 'LANDSAT_NDWI_ICE', 0.30)
static_threshol_25_eotask = StaticThreshiolding('WATER_MASK_ICE_ST_025', 'LANDSAT_NDWI_ICE', 0.25)
static_threshol_20_eotask = StaticThreshiolding('WATER_MASK_ICE_ST_020', 'LANDSAT_NDWI_ICE', 0.20)
static_threshol_15_eotask = StaticThreshiolding('WATER_MASK_ICE_ST_015', 'LANDSAT_NDWI_ICE', 0.15)
static_threshol_10_eotask = StaticThreshiolding('WATER_MASK_ICE_ST_010', 'LANDSAT_NDWI_ICE', 0.10)


for date_range_landsat in site.DATE_RANGES_LANDSAT_8 :
    print('Reading EOPatches from:', date_range_landsat )
    eopatches_filepath = io_utils.get_eopatches_dir(site.DATA_PRODUCTS[1], site.SITE_NAME, site.AVAILABLE_CRSs[0], date_range_landsat)
    # TASK TO LOAD AND SAVE EXISTING EOPATCHES HAS TO BE CREATED YEARLY (different directories)
    load_eotask = LoadTask(eopatches_filepath)
    save_eotask = SaveTask(eopatches_filepath, overwrite_permission=OverwritePermission.OVERWRITE_PATCH, compress_level=1)
    
    # Define the workflow
    workflow = LinearWorkflow(
    load_eotask,
    static_threshol_30_eotask,
    static_threshol_25_eotask,
    static_threshol_20_eotask,
    static_threshol_15_eotask,
    static_threshol_10_eotask,
    save_eotask
    )
                      
    list_of_available_patches = io_utils.get_list_of_eopatches(eopatches_filepath)
    execution_args = []
    for eopatch_name in list_of_available_patches:
        execution_args.append({
        load_eotask: {'eopatch_folder': eopatch_name},
        save_eotask: {'eopatch_folder': eopatch_name}
        })
    
    #print(execution_args)                            
    executor = EOExecutor(workflow, execution_args, save_logs=True)
    executor.run(workers=1, multiprocess=False)

    executor.make_report()

Reading EOPatches from: ('2013-05-01', '2013-10-31')


C:\Users\Kamil\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in greater_equal
  from ipykernel import kernelapp as app


Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created ne

Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected

Reading EOPatches from: ('2015-05-01', '2015-10-31')


Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected

Reading EOPatches from: ('2016-05-01', '2016-10-31')


Created new mask of zeros, bc there was no water feature detected

Reading EOPatches from: ('2017-05-01', '2017-10-31')


Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected

Reading EOPatches from: ('2018-05-01', '2018-10-31')


Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected

Reading EOPatches from: ('2019-05-01', '2019-10-31')


Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected



In [9]:
# creating eo tasks producing water mask from NDWI (based on green/NIR NDI)
static_threshol_30_eotask = StaticThreshiolding('WATER_MASK_ST_030', 'LANDSAT_NDWI', 0.30)
static_threshol_25_eotask = StaticThreshiolding('WATER_MASK_ST_025', 'LANDSAT_NDWI', 0.25)
static_threshol_20_eotask = StaticThreshiolding('WATER_MASK_ST_020', 'LANDSAT_NDWI', 0.20)
static_threshol_15_eotask = StaticThreshiolding('WATER_MASK_ST_015', 'LANDSAT_NDWI', 0.15)
static_threshol_10_eotask = StaticThreshiolding('WATER_MASK_ST_010', 'LANDSAT_NDWI', 0.10)


for date_range_landsat in site.DATE_RANGES_LANDSAT_8 :
    print('Reading EOPatches from:', date_range_landsat )
    eopatches_filepath = io_utils.get_eopatches_dir(site.DATA_PRODUCTS[1], site.SITE_NAME, site.AVAILABLE_CRSs[0], date_range_landsat)
    # TASK TO LOAD AND SAVE EXISTING EOPATCHES HAS TO BE CREATED YEARLY (different directories)
    load_eotask = LoadTask(eopatches_filepath)
    save_eotask = SaveTask(eopatches_filepath, overwrite_permission=OverwritePermission.OVERWRITE_PATCH, compress_level=1)
    
    # Define the workflow
    workflow = LinearWorkflow(
    load_eotask,
    static_threshol_30_eotask,
    static_threshol_25_eotask,
    static_threshol_20_eotask,
    static_threshol_15_eotask,
    static_threshol_10_eotask,
    save_eotask
    )
                      
    list_of_available_patches = io_utils.get_list_of_eopatches(eopatches_filepath)
    execution_args = []
    for eopatch_name in list_of_available_patches:
        execution_args.append({
        load_eotask: {'eopatch_folder': eopatch_name},
        save_eotask: {'eopatch_folder': eopatch_name}
        })
    
    #print(execution_args)                            
    executor = EOExecutor(workflow, execution_args, save_logs=True)
    executor.run(workers=1, multiprocess=False)

    executor.make_report()

Reading EOPatches from: ('2013-05-01', '2013-10-31')


Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created ne

Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected

Reading EOPatches from: ('2015-05-01', '2015-10-31')



Reading EOPatches from: ('2016-05-01', '2016-10-31')


Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected

Reading EOPatches from: ('2017-05-01', '2017-10-31')


Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected

Reading EOPatches from: ('2018-05-01', '2018-10-31')


Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected

Reading EOPatches from: ('2019-05-01', '2019-10-31')


Created new mask of zeros, bc there was no water feature detected

